This notebook is part of [**Byron v0.8**](https://pypi.org/project/byron/)  
Copyright 2023-24 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: [Apache-2.0](https://opensource.org/license/apache-2-0/) 

In [1]:
# Did you `!pip install -U byron`?
import byron

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_6024/670150310.py:2: ByronPerformanceWarning: 
  Paranoia checks are enabled in this notebook: performances can be significantly impaired
  [see https://cad-polito-it.github.io/byron/paranoia for details]
  import byron


In [2]:
byron.welcome()

This is Byron v0.8a1 "Don Juan" (dev62 @ 24-08-2024)
Copyright (c) 2023-24 Giovanni Squillero and Alberto Tonda

# Macros & Parameters

## Parameters

Everything for the definition of the constraints (the old "instruction library") is under the namespace `byron.framework` (lazy fingers can use `byron.f`)

In [3]:
import math

byte = byron.f.integer_parameter(0, 2 ** 8)
almost_pi = byron.f.float_parameter(21 * math.sin(math.pi / 21), 42 * math.tan(math.pi / 42))
register = byron.f.choice_parameter(['ax', 'bx', 'cx', 'dx'])
bitmask = byron.f.array_parameter("01-", 32)

Parameters are *types* (classes), that is, all the above functions are technically *factories*. 

In [4]:
byte

Int[8bit]

Parameters are used by Byron to create objects (*instances*). The initial (non initialized) value of these objects is always `None`; parameter instances are initialized with the first mutation and then, possibly, mutated again.

**Notez bien:** Users do not need to use *parameters* directly. 

In [5]:
for param in [byte, almost_pi, register, bitmask]:
    obj = param()
    initial_value = obj.value
    obj.mutate()
    value = obj.value
    print(f"{obj!r} :: {initial_value!r} -> {value!r}")

<byron.parameter.Int[8bit] at 0x115f996a0> :: None -> 57
<byron.parameter.Float[3.1298875896996634–3.1474648808016528) at 0x115f996d0> :: None -> 3.130327211024893
<byron.parameter.Choice[ax┊bx┊cx┊dx] at 0x115f996a0> :: None -> 'cx'
<byron.parameter.Array[-01ｘ32] at 0x115f7fd40> :: None -> '---1-111-10-----11-1-11110-010--'


Instances of `byron.parameter` can be printed using f-string [syntax](https://docs.python.org/3/reference/lexical_analysis.html#f-strings)

In [6]:
num = byte()
num.mutate()
print(f"Default: {num}\nRepr: {num!r}\nBinary (16bit): {num:016b}\nOctal: {num:o}\n#Hex: {num:#x}")

Default: 216
Repr: <byron.parameter.Int[8bit] at 0x115bc5520>
Binary (16bit): 0000000011011000
Octal: 330
#Hex: 0xd8


The type of a parameter and some sample values may be visualized using *as_text*

In [7]:
byron.f.as_text(almost_pi)

Float[3.1298875896996634–3.1474648808016528)
• 3.1411269806745707
• 3.130327211024893
• 3.134721860090219
• 3.133811029821195
• 3.1428327586192597
• 3.141782133578663
• 3.1455696896764587
• 3.1314157388688577




## Aliases (shared parameters) 

Parameters are classes (types), thus objects created are independent. 

In [8]:
parameter = byron.f.integer_parameter(0, 10_000)

var1 = parameter()
var2 = parameter()
var3 = parameter()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  Initial")
var1.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var1")
var2.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var2")
var3.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var3")

var1: None // var2: None // var3: None  --  Initial
var1: 5574 // var2: None // var3: None  --  After mutating var1
var1: 5574 // var2: 4552 // var3: None  --  After mutating var2
var1: 5574 // var2: 4552 // var3: 2547  --  After mutating var3


However, it may be necessary to have all instances of a specific parameter sharing the very same data. Parameters defined by a `make_shared_parameter` behave like *aliases*: they always have the same value, changes (e.g., `mutate()`) only affect the *first* instance.

In [9]:
shared_parameter1 = byron.f.make_shared_parameter(byron.f.integer_parameter(0, 10_000))
shared_parameter2 = byron.f.make_shared_parameter(byron.f.integer_parameter(0, 10_000))

var1 = shared_parameter1()
var2 = shared_parameter1()
var3 = shared_parameter2()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  Initial")
var1.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var1")
var2.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var2")
var3.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var3")

var1: None // var2: None // var3: None  --  Initial
var1: 3527 // var2: 3527 // var3: None  --  After mutating var1
var1: 3527 // var2: 3527 // var3: None  --  After mutating var2
var1: 3527 // var2: 3527 // var3: 5514  --  After mutating var3


## Notes

### Beautiful and Meaningful names

In [10]:
p1 = byron.f.integer_parameter(0, 100)
p2 = byron.f.integer_parameter(0, 256)
p3 = byron.f.choice_parameter("BAGFECD")

print(f"{p1!r} / {p1()!r}")
print(f"{p2!r} / {p2()!r}")
print(f"{p3!r} / {p3()!r}")

Int[0..99] / <byron.parameter.Int[0..99] at 0x115f981d0>
Int[8bit] / <byron.parameter.Int[8bit] at 0x115f9be60>
Choice[A┊B┊C┊D┊E┊F┊G] / <byron.parameter.Choice[A┊B┊C┊D┊E┊F┊G] at 0x115f99ca0>


### Helpful warnings

Friendly suggestions are shown if code is not optimized and logging level is `DEBUG` (the default in notebooks)

In [11]:
possible_error = byron.f.integer_parameter(1, 100)

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_6024/2880333942.py:1: ByronFriendlyWarning: 
  Parameter ranges are half-open: the maximum value is 99 (ie. a range of 99 possible values)
  possible_error = byron.f.integer_parameter(1, 100)


In [12]:
possible_error = byron.f.choice_parameter(range(1_000))

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_6024/3768838695.py:1: ByronFriendlyWarning: 
  Choice parameters with many alternatives impair performances — why not using an integer parameter [0-1000)?
  possible_error = byron.f.choice_parameter(range(1_000))


### Cache

Results are cached. Calling the same parameter factory with the same arguments yields the same parameter (the **very same** one, not an identical copy)

In [13]:
word1 = byron.f.integer_parameter(0, 2 ** 16)
word2 = byron.f.integer_parameter(0, 65_536)

word1 == word2, word1 is word2

(True, True)

In [14]:
choice1 = byron.f.choice_parameter(['Yes', 'No', 'Maybe'])
choice2 = byron.f.choice_parameter(('Maybe', 'Yes', 'No'))

choice1 == choice2, choice1 is choice2

(True, True)

# Macros

Macros represent multi-line parametric fragments. The text specifies placeholders using the [f-string syntax](https://peps.python.org/pep-0498/), then parameters are defined as key arguments. Macros, as a Frames, can be tested using `show`.

In [15]:
macro = byron.f.macro(
    "{var} := {num}", var=byron.f.choice_parameter(['foo', 'bar']), num=byron.f.integer_parameter(-100, 100)
)
byron.f.as_text(macro)

bar := 63  ; 🖋 n2 ➜ Macro❬User#2❭



In [16]:
variable = byron.f.choice_parameter(['foo', 'bar'])
integer = byron.f.integer_parameter(-100, 100)
macro = byron.f.macro("{var} := {num:#x} ; ie. set the variable '{var}' to {num}", var=variable, num=integer)
byron.f.as_text(macro)

bar := 0x3f ; ie. set the variable 'bar' to 63  ; 🖋 n3 ➜ Macro❬User#3❭



## Extra Parameters

Macros can use extra parameters that are defined by Byron. The names of all extra parameters start with an underscore. 

The extra parameter `_node` is a lazy `NodeView` that provides information about the node the macro is in. Through the `_node` it is possible to access nearly all internal data structures.

In [17]:
macro = byron.f.macro('{_node}\n\n{_node!r}\n\n{_node.fields}')
byron.f.as_text(macro, node_info=False)

n4

NodeView(ref=NodeReference(graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x115fdb860>, node=n4))

['children', 'fields', 'graph', 'make', 'node', 'node_attributes', 'node_type', 'out_degree', 'p', 'parent', 'path', 'path_string', 'ref', 'safe_dump', 'selement', 'tree', 'type_']



the `p` field of the NodeView contains a `ValueBag` with macro parameters (if any). Please remember that *ValueBag*s are reasonably safe: missing items default to `None` (e.g., `value_bag.huitzilopochtli` would be `None` except for few users fan of Aztec culture).

In [18]:
macro = byron.f.macro(
    'x={x} y="{y}" z={z}\n\n{_node.p}',
    x=byron.f.integer_parameter(0, 2 ** 8),
    y=byron.f.choice_parameter("abc"),
    z=byron.f.float_parameter(0, 1),
)
byron.f.as_text(macro, node_info=False)

x=57 y="a" z=0.7415504997598329

{{'x': 57, 'y': 'a', 'z': 0.7415504997598329}}



Other extra paramet control how the individual is dumped into a string

In [19]:
byron.DEFAULT_EXTRA_PARAMETERS

{'_comment': ';',
 '_label': '{_node}:\n',
 '_text_before_macro': '',
 '_text_after_macro': '\n',
 '_text_before_frame': '',
 '_text_after_frame': '',
 '_text_before_node': '',
 '_text_after_node': ''}

In [20]:
byron.f.set_global_parameter('_comment', '#')
byron.f.as_text(byron.f.macro('Frank'))

byron.f.as_text(byron.f.macro('Zappa', _comment='@'))

Frank  # 🖋 n6 ➜ Macro❬Text#2❭

Zappa  @ 🖋 n7 ➜ Macro❬Text#3❭



# *as_text* (*as_lgp* and *as_forest*)

In [21]:
macro = byron.f.macro("{var} := {num}", var=byron.f.choice_parameter("abcde"), num=byron.f.integer_parameter(0, 1000))

In [22]:
for _ in range(5):
    byron.f.as_text(macro)

a := 654  # 🖋 n8 ➜ Macro❬User#5❭

a := 654  # 🖋 n9 ➜ Macro❬User#5❭

a := 654  # 🖋 n10 ➜ Macro❬User#5❭

a := 654  # 🖋 n11 ➜ Macro❬User#5❭

a := 654  # 🖋 n12 ➜ Macro❬User#5❭



In [23]:
for _ in range(5):
    byron.f.as_text(macro, node_info=False, seed=2310)

e := 309

e := 309

e := 309

e := 309

e := 309



In [24]:
for _ in range(5):
    byron.f.as_text(macro, node_info=False, seed=None)

d := 994

b := 462

d := 146

d := 790

b := 670

